In [ ]:
!pip install Path
!pip install pyspark

In [62]:
from pyspark import SparkContext, SparkConf 



In [65]:
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate(conf=conf)

In [11]:
# Adjacency list
rdd = sc.textFile("/content/drive/MyDrive/link_data.txt")
rdd.collect()

['0 2', '2 0', '1 2', '1 3', '3 2']

In [ ]:
links = rdd.map(lambda urls: parseNeighbors(urls))  

In [72]:
# Key/value pairs
rdd1 = rdd.map(lambda x: (x.split(' ')[0], x.split(' ')[1:])).reduceByKey(lambda x, y: x+y)
print(rdd1.collect())

[('0', ['2']), ('2', ['0']), ('1', ['2', '3']), ('3', ['2'])]


In [73]:
# Find node count
N = rdd1.count()
print(N)

4


In [76]:
# Create and initialize the ranks
ranks = rdd1.map(lambda x: (x[0], 1.0))
print(ranks.collect())

[('0', 1.0), ('2', 1.0), ('1', 1.0), ('3', 1.0)]


In [75]:
ITERATIONS=20
for i in range(5):
    # calculate the contribution of each link
    contribs = rdd1.join(ranks).flatMap(lambda x : [(i, float(x[1][1])/len(x[1][0])) for i in x[1][0]])

    # update each rank by summing up all incoming link's contribution
    ranks = contribs.reduceByKey(lambda x,y: 0.05 + 0.85 + 0.10)

    # check progress
    print(ranks.sortByKey().collect())

[('0', 1.0), ('2', 1.0), ('3', 0.5)]
[('0', 1.0), ('2', 1.0)]
[('0', 1.0), ('2', 1.0)]
[('0', 1.0), ('2', 1.0)]
[('0', 1.0), ('2', 1.0)]


In [70]:
for (link, rank) in sorted(ranks.collect()):
    print("%s has rank: %s." % (link, rank))

0 has rank: 1.0.
2 has rank: 1.0.
